In [18]:
import re
from pathlib import Path
import pandas as pd
import numpy as np
from openpyxl.chart import BarChart, Reference
import re, uuid
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import Alignment, Font, PatternFill
from openpyxl.utils import get_column_letter

# ===== CONFIG =====
ONLY_PAID = False   # True = solo facturas pagadas, False = incluir todas
INPUT_PATTERN = r"Maria Vert Carbó - Ingresos \d{2}_\d{2}_\d{4}-\d{2}_\d{2}_\d{4}\.xlsx"
OUTPUT_FILENAME = "ingresos_report_global.xlsx"

# Product category mapping (substring -> Category), order matters
PRODUCT_CATEGORY_RULES = [
    ("nuvies",       "Novias"),
    ("complements",  "Complementos"),
    ("premsades",    "Premsades"),
    ("decoracions",  "Decoracions"),
    ("b2b",          "B2B"),
    ("florece",      "Formaciones"),
    ("formacion",    "Formaciones"),  # extra mapping you requested
]
PRODUCT_DEFAULT = "Productos web, cuadros, cúpulas y varios"

# Sales channel mapping (substring -> Channel), order matters
CHANNEL_RULES = [
    ("instagram",   "Instagram"),
    ("#ig",         "Instagram"),
    ("tiktok",      "TikTok"),
    ("google",      "Google"),
    ("madrigal",    "Madrigal"),
    ("referencies", "Referencias"),
    ("proveidors",  "Proveedores"),
]
CHANNEL_DEFAULT = "Sin identificar"
CHANNEL_PRODUCT_ONLY = "No aplica (solo producto)"

AMOUNT_CANDIDATES = ["Total", "Importe", "Amount", "Ingreso", "Income"]

# Phrases to detect in Descripción (case-insensitive, accent-safe)
PHRASE_RAMO     = "pago a cuenta ramo de novia"
PHRASE_PRENSADO = "pago a cuenta cuadro prensado"

# ===== HELPERS =====
def _first_sheet_name(xls_path: Path) -> str:
    xfile = pd.ExcelFile(xls_path)
    if not xfile.sheet_names:
        raise ValueError("This workbook has no sheets.")
    return xfile.sheet_names[0]

def find_header_row(xls_path: Path, sheet_name: str | None = None) -> int:
    if sheet_name is None:
        sheet_name = _first_sheet_name(xls_path)
    df_raw = pd.read_excel(xls_path, sheet_name=sheet_name, header=None, dtype=str)
    for i in range(min(40, len(df_raw))):
        row_vals = df_raw.iloc[i].astype(str).str.strip().str.lower().tolist()
        if ("tags" in row_vals) or ("fecha emisión" in row_vals) or ("fecha emision" in row_vals):
            return i
    return 0

def load_table(xls_path: Path, sheet_name: str | None = None) -> pd.DataFrame:
    if sheet_name is None:
        sheet_name = _first_sheet_name(xls_path)
    header_row = find_header_row(xls_path, sheet_name)
    df = pd.read_excel(xls_path, sheet_name=sheet_name, header=header_row)
    df.columns = [str(c).strip() for c in df.columns]
    return df.dropna(how="all")

def pick_amount_column(df: pd.DataFrame) -> str:
    for cand in AMOUNT_CANDIDATES:
        if cand in df.columns:
            return cand
    for c in df.columns:  # last resort: coerce and pick first numeric-ish
        coerced = pd.to_numeric(df[c], errors="coerce")
        if coerced.notna().any():
            df[c] = coerced
            return c
    raise ValueError("No numeric amount column found.")

def _first_match_from_rules(text: str, rules):
    t = text.lower()
    for sub, label in rules:
        if sub.lower() in t:
            return label
    return None

def _has_any_keyword(text: str, rules) -> bool:
    t = text.lower()
    return any(sub.lower() in t for sub, _ in rules)

def map_product(tags_val) -> str:
    if pd.isna(tags_val):
        return PRODUCT_DEFAULT
    label = _first_match_from_rules(str(tags_val), PRODUCT_CATEGORY_RULES)
    return label if label is not None else PRODUCT_DEFAULT

def map_channel(tags_val) -> str:
    if pd.isna(tags_val):
        return CHANNEL_DEFAULT
    text = str(tags_val)
    ch_label = _first_match_from_rules(text, CHANNEL_RULES)
    if ch_label is not None:
        return ch_label
    if _has_any_keyword(text, PRODUCT_CATEGORY_RULES):
        return CHANNEL_PRODUCT_ONLY
    return CHANNEL_DEFAULT

def month_label_from_filename(filename: str) -> str:
    m = re.search(r"Ingresos (\d{2})_(\d{2})_(\d{4})-(\d{2})_(\d{2})_(\d{4})", filename)
    if m:
        dd, mm, yyyy = m.group(1), m.group(2), m.group(3)
        return f"{yyyy}-{mm}"
    return Path(filename).stem[:31]

def unique_labels_in_order(rules, extra_last: str | None = None) -> list[str]:
    seen, out = set(), []
    for _, lab in rules:
        if lab not in seen:
            out.append(lab); seen.add(lab)
    if extra_last and extra_last not in seen:
        out.append(extra_last)
    return out

def unique_labels_in_order_from_rules(rules, extras=()):
    """Return unique labels from (substring, label) rules preserving order.
       Optionally append extra labels (also deduped)."""
    seen, out = set(), []
    for _, lab in rules:
        if lab not in seen:
            out.append(lab); seen.add(lab)
    for lab in extras:
        if lab not in seen:
            out.append(lab); seen.add(lab)
    return out


def map_pago_tipo(desc):
    s = str(desc).casefold()
    if PHRASE_RAMO in s:
        return "Pago a cuenta ramo de novia"
    if PHRASE_PRENSADO in s:
        return "Pago a cuenta cuadro prensado"
    return None

def build_pagos_counts(df_clean: pd.DataFrame, all_channels: list[str]) -> pd.DataFrame:
    if "Descripción" not in df_clean.columns:
        # If not available, return an empty skeleton with channels and zeros
        counts = pd.DataFrame(index=all_channels, columns=[
            "Pago a cuenta ramo de novia", "Pago a cuenta cuadro prensado"
        ]).fillna(0).astype(int)
        counts["Total"] = counts.sum(axis=1)
        totals_row = counts.sum(axis=0).to_frame().T
        totals_row.index = ["Total"]
        return pd.concat([counts, totals_row], axis=0)
    tmp = df_clean.copy()
    tmp["Tipo"] = tmp["Descripción"].apply(map_pago_tipo)
    tmp = tmp[tmp["Tipo"].notna()].copy()
    if tmp.empty:
        counts = pd.DataFrame(index=all_channels, columns=[
            "Pago a cuenta ramo de novia", "Pago a cuenta cuadro prensado"
        ]).fillna(0).astype(int)
    else:
        counts = (tmp.groupby(["Canal", "Tipo"]).size()
                    .unstack(fill_value=0)
                    .reindex(index=all_channels, columns=[
                        "Pago a cuenta ramo de novia", "Pago a cuenta cuadro prensado"
                    ], fill_value=0)
                 )
    counts["Total"] = counts.sum(axis=1)
    totals_row = counts.sum(axis=0).to_frame().T
    totals_row.index = ["Total"]
    return pd.concat([counts, totals_row], axis=0)


def _safe_table_name(base: str) -> str:
    """
    Excel table names:
      - Only letters, digits, underscore
      - Must start with a letter
      - Keep it short (some Excel builds choke on very long names)
    """
    s = re.sub(r'[^A-Za-z0-9_]', '_', base)
    if not s or not s[0].isalpha():
        s = "T_" + s
    # Keep it short and unique
    suffix = uuid.uuid4().hex[:6]
    s = s[:20] + "_" + suffix   # final length ~ 27 chars
    return s

def _add_styled_table(ws, df, start_row, start_col=1, name_prefix="T"):
    # Write the header row (always)
    for j, col in enumerate(df.columns, start=start_col):
        cell = ws.cell(row=start_row, column=j, value=col)
        cell.font = Font(bold=True)
        cell.alignment = Alignment(horizontal="center")
        cell.fill = PatternFill("solid", fgColor="F3F6FA")

    # Write data rows (if any)
    for i, (_, row) in enumerate(df.iterrows(), start=start_row + 1):
        for j, val in enumerate(row, start=start_col):
            ws.cell(row=i, column=j, value=val)

    nrows = len(df) + 1
    ncols = len(df.columns)
    end_row = start_row + nrows - 1
    end_col = start_col + ncols - 1

    # If there are no data rows, don't create an Excel Table (Excel will "repair" it)
    if len(df) == 0 or ncols == 0:
        return end_row, end_col

    # Build the ref and a safe, unique table name
    ref = f"{get_column_letter(start_col)}{start_row}:{get_column_letter(end_col)}{end_row}"
    base_name = f"{name_prefix}_{ws.title}_{start_row}_{start_col}"
    table_name = _safe_table_name(base_name)

    table = Table(displayName=table_name, ref=ref)
    table.tableStyleInfo = TableStyleInfo(
        name="TableStyleMedium9", showRowStripes=True, showColumnStripes=False
    )
    ws.add_table(table)

    return end_row, end_col



def _number_format_columns(ws, start_row, end_row, start_col, headers):
    """
    Apply number formats based on header names.
    - 'Total' -> thousands separator
    - 'Porcentaje' -> keep as 2 decimals (because values are already 0–100)
    """
    for j, h in enumerate(headers, start=start_col):
        if str(h).lower().startswith("total"):
            nf = '#,##0'
        elif "porcentaje" in str(h).lower():
            nf = '0.00'
        else:
            nf = None
        if nf:
            for r in range(start_row+1, end_row+1):
                ws.cell(row=r, column=j).number_format = nf

def _autofit_columns(ws, first_row=1, last_row=None, margin=2):
    """Approximate auto-fit by measuring text length in each column."""
    if last_row is None:
        last_row = ws.max_row
    for col in range(1, ws.max_column + 1):
        max_len = 0
        for row in range(first_row, last_row + 1):
            v = ws.cell(row=row, column=col).value
            if v is None:
                continue
            v = str(v)
            # give a bit more space for header (bold font is wider)
            if row == first_row:
                max_len = max(max_len, len(v) + 2)
            else:
                max_len = max(max_len, len(v))
        ws.column_dimensions[get_column_letter(col)].width = max(8, min(max_len + margin, 60))


# ===== CORE aggregation for one DataFrame =====
def analyze_df(df: pd.DataFrame, amount_col: str):
    # Filter paid if requested and column exists
    if ONLY_PAID and "Estado" in df.columns:
        df = df[df["Estado"].astype(str).str.lower().str.strip() == "pagado"].copy()

    if "Tags" not in df.columns:
        raise ValueError("The Excel does not contain a 'Tags' column.")

    df[amount_col] = pd.to_numeric(df[amount_col], errors="coerce")

    df["Categoría"] = df["Tags"].apply(map_product)
    df["Canal"]     = df["Tags"].apply(map_channel)

    ALL_CATEGORIES = unique_labels_in_order(PRODUCT_CATEGORY_RULES, PRODUCT_DEFAULT)
    #ALL_CHANNELS   = [lab for _, lab in CHANNEL_RULES] + [CHANNEL_PRODUCT_ONLY, CHANNEL_DEFAULT]
    #ALL_CHANNELS = unique_labels_in_order(CHANNEL_RULES, CHANNEL_DEFAULT)
    ALL_CHANNELS = unique_labels_in_order_from_rules(
    CHANNEL_RULES,
    extras=(CHANNEL_PRODUCT_ONLY, CHANNEL_DEFAULT)
)


    grand_total = float(pd.to_numeric(df[amount_col], errors="coerce").sum())
    if grand_total == 0:
        grand_total = np.nan

    tot_cat = (
        df.groupby("Categoría", dropna=False)[amount_col]
          .sum(min_count=1).fillna(0)
          .reindex(ALL_CATEGORIES)
          .reset_index()
          .rename(columns={amount_col: "Total"})
    )
    tot_cat["Porcentaje"] = (tot_cat["Total"] / grand_total * 100).round(2)

    tot_channel = (
        df.groupby("Canal", dropna=False)[amount_col]
          .sum(min_count=1).fillna(0)
          .reindex(ALL_CHANNELS)
          .reset_index()
          .rename(columns={amount_col: "Total"})
    )
    tot_channel["Porcentaje"] = (tot_channel["Total"] / grand_total * 100).round(2)

    pivot = (
        df.groupby(["Canal", "Categoría"], dropna=False)[amount_col]
          .sum(min_count=1).fillna(0)
          .unstack(fill_value=0)
    ).reindex(index=ALL_CHANNELS, columns=ALL_CATEGORIES, fill_value=0)


    pivot_with_totals = pivot.copy()
    pivot_with_totals["Total canal (€)"] = pivot_with_totals.sum(axis=1)
    row_total = pd.DataFrame([pivot_with_totals.sum(axis=0)], index=["Total (€)"])
    pivot_with_totals = pd.concat([pivot_with_totals, row_total], axis=0)

    # ---- Matriz de % con totales coherentes (Opción B) ----
    # % por celda sobre el TOTAL global
    pivot_pct = (pivot / grand_total * 100)

    # Totales reales por canal y por categoría (también % sobre el TOTAL global)
    channel_totals_pct  = (pivot.sum(axis=1) / grand_total * 100)
    category_totals_pct = (pivot.sum(axis=0) / grand_total * 100)

    # Construimos una copia con la nueva columna de total por canal
    pivot_pct_with_totals = pivot_pct.copy()
    pivot_pct_with_totals["Total canal (%)"] = channel_totals_pct

    # Añadimos la fila de total por categoría
    row_total_pct = pd.DataFrame([category_totals_pct], index=["Total (%)"])

    # También añadimos el valor 100% en la esquina inferior derecha
    row_total_pct["Total canal (%)"] = 100.0

    # Concatenamos fila final
    pivot_pct_with_totals = pd.concat([pivot_pct_with_totals, row_total_pct], axis=0)

    # Redondeo al final
    pivot_pct_with_totals = pivot_pct_with_totals.round(2)


    # Build pagos-a-cuenta counts here, once
    pagos_counts = build_pagos_counts(df, ALL_CHANNELS)

    return {
        "tot_cat": tot_cat.sort_values("Total", ascending=False),
        "tot_channel": tot_channel.sort_values("Total", ascending=False),
        "pivot_eur": pivot_with_totals,
        "pivot_pct": pivot_pct_with_totals,
        "df_clean": df,
        "all_channels": ALL_CHANNELS,
        "pagos_counts": pagos_counts,
    }

# ===== WRITE one month to ONE sheet (stacked tables + two charts + pagos) =====
def write_month_sheet(writer, month_name: str, results: dict):
    ws_name = month_name[:31]
    ws = writer.book.create_sheet(ws_name)  # create first so we can format freely
    writer.sheets[ws_name] = ws
    ws.freeze_panes = "A2"

    row = 1

    # 1) Categoría table + chart
    end_row, end_col = _add_styled_table(ws, results["tot_cat"], start_row=row, start_col=1, name_prefix="CAT")
    _number_format_columns(ws, start_row=row, end_row=end_row, start_col=1, headers=list(results["tot_cat"].columns))

    from openpyxl.chart import BarChart, Reference
    chart_cat = BarChart()
    chart_cat.title = "Ingresos por categoría"
    chart_cat.y_axis.title = "€"
    chart_cat.x_axis.title = "Categoría"
    data_cat = Reference(ws, min_col=2, min_row=row, max_col=2, max_row=end_row)
    cats_cat = Reference(ws, min_col=1, min_row=row+1, max_row=end_row)
    chart_cat.add_data(data_cat, titles_from_data=True)
    chart_cat.set_categories(cats_cat)
    chart_cat.width = 18    # in "Excel" units (roughly cm)
    chart_cat.height = 10
    # Place far to the right so it never hits the next table
    ws.add_chart(chart_cat, "K2")

    row = end_row + 3

    # 2) Canales table + chart
    end_row, end_col = _add_styled_table(ws, results["tot_channel"], start_row=row, start_col=1, name_prefix="CHN")
    _number_format_columns(ws, start_row=row, end_row=end_row, start_col=1, headers=list(results["tot_channel"].columns))

    chart_chn = BarChart()
    chart_chn.title = "Ingresos por canal"
    chart_chn.y_axis.title = "€"
    chart_chn.x_axis.title = "Canal"
    data_chn = Reference(ws, min_col=2, min_row=row, max_col=2, max_row=end_row)
    cats_chn = Reference(ws, min_col=1, min_row=row+1, max_row=end_row)
    chart_chn.add_data(data_chn, titles_from_data=True)
    chart_chn.set_categories(cats_chn)
    chart_chn.width = 18
    chart_chn.height = 10
    # Place this **below** the first chart to be extra safe
    ws.add_chart(chart_chn, "K23")

    row = end_row + 3

    # 3) Pivot € table
    end_row, end_col = _add_styled_table(ws, results["pivot_eur"].reset_index(), start_row=row, start_col=1, name_prefix="PIV_EUR")
    _number_format_columns(ws, start_row=row, end_row=end_row, start_col=1, headers=list(results["pivot_eur"].reset_index().columns))
    row = end_row + 3

    # 4) Pivot % table
    end_row, end_col = _add_styled_table(ws, results["pivot_pct"].reset_index(), start_row=row, start_col=1, name_prefix="PIV_PCT")
    _number_format_columns(ws, start_row=row, end_row=end_row, start_col=1, headers=list(results["pivot_pct"].reset_index().columns))
    row = end_row + 3

    # 5) Pagos a cuenta (conteo)
    end_row, end_col = _add_styled_table(ws, results["pagos_counts"], start_row=row, start_col=1, name_prefix="PAGOS")

    # Final tidy-up: autofit all columns
    _autofit_columns(ws, first_row=1, last_row=ws.max_row)


# ===== MAIN: process all files and build TOTAL =====
files = sorted([p for p in Path(".").glob("*.xlsx") if re.fullmatch(INPUT_PATTERN, p.name)])
if not files:
    raise FileNotFoundError("No monthly files found matching the expected pattern in the current directory.")

monthly_results = []
all_rows = []

for f in files:
    df = load_table(f)
    amount_col = pick_amount_column(df)
    res = analyze_df(df, amount_col)
    monthly_results.append((month_label_from_filename(f.name), res))
    all_rows.append(res["df_clean"])

# TOTAL across all months
df_all = pd.concat(all_rows, ignore_index=True)
amount_col_all = pick_amount_column(df_all)
total_res = analyze_df(df_all, amount_col_all)

# ===== Write everything to workbook =====
output_path = Path(OUTPUT_FILENAME)
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    for month_name, res in monthly_results:
        write_month_sheet(writer, month_name, res)
    write_month_sheet(writer, "TOTAL", total_res)
    
    # ===== Tendencias (time series) =====
    ws = writer.book.create_sheet("Tendencias")
    writer.sheets["Tendencias"] = ws
    ws.freeze_panes = "A2"

    # Build monthly trend tables
    target_categories = ["Novias", "Premsades", "Formaciones"]
    target_channels   = ["TikTok", "Instagram", "Google"]
    months_order = [month for month, _ in monthly_results]

    def _sum_for_label(df, label_col, label, amount_col_name):
        if label not in df[label_col].unique():
            return 0.0
        return pd.to_numeric(df.loc[df[label_col] == label, amount_col_name], errors="coerce").sum()

    # Initialize dataframes
    cat_trend_df = pd.DataFrame(0.0, index=months_order, columns=target_categories)
    chn_trend_df = pd.DataFrame(0.0, index=months_order, columns=target_channels)

    # Fill with monthly totals
    for month_name, res in monthly_results:
        dfm = res["df_clean"]
        amt_col_m = pick_amount_column(dfm)
        for cat in target_categories:
            cat_trend_df.loc[month_name, cat] = _sum_for_label(dfm, "Categoría", cat, amt_col_m)
        for ch in target_channels:
            chn_trend_df.loc[month_name, ch] = _sum_for_label(dfm, "Canal", ch, amt_col_m)

    # Add TOTAL line (sum across the selected series)
    cat_trend_df["TOTAL"] = cat_trend_df.sum(axis=1)
    chn_trend_df["TOTAL"] = chn_trend_df.sum(axis=1)

    # 1) Categorías table + chart (with TOTAL in black)
    row = 1
    from openpyxl.chart import LineChart, Reference
    end_row, end_col = _add_styled_table(
        ws,
        cat_trend_df.reset_index().rename(columns={"index": "Mes"}),
        row, 1, "TREND_CAT"
    )

    # include all columns (including TOTAL)
    num_cat_cols = 1 + len(cat_trend_df.columns)  # 1 for 'Mes' col in sheet
    data_ref = Reference(ws, min_col=2, min_row=row, max_col=num_cat_cols, max_row=end_row)
    cats_ref = Reference(ws, min_col=1, min_row=row+1, max_row=end_row)

    chart1 = LineChart()
    chart1.title = "Tendencia mensual por categoría"
    chart1.y_axis.title = "Ingresos (€)"
    chart1.x_axis.title = "Mes"
    chart1.add_data(data_ref, titles_from_data=True)
    chart1.set_categories(cats_ref)
    # make TOTAL line black & thicker
    for s in chart1.series:
        if getattr(s.title, "value", None) == "TOTAL":
            s.graphicalProperties.line.solidFill = "000000"
            s.graphicalProperties.line.width = 30000  # thicker
    chart1.width = 20; chart1.height = 12
    ws.add_chart(chart1, "G2")
    row = end_row + 3

    # 2) Canales table + chart (with TOTAL in black)
    end_row, end_col = _add_styled_table(
        ws,
        chn_trend_df.reset_index().rename(columns={"index": "Mes"}),
        row, 1, "TREND_CHN"
    )

    num_chn_cols = 1 + len(chn_trend_df.columns)
    data_ref2 = Reference(ws, min_col=2, min_row=row, max_col=num_chn_cols, max_row=end_row)
    cats_ref2 = Reference(ws, min_col=1, min_row=row+1, max_row=end_row)

    chart2 = LineChart()
    chart2.title = "Tendencia mensual por canal"
    chart2.y_axis.title = "Ingresos (€)"
    chart2.x_axis.title = "Mes"
    chart2.add_data(data_ref2, titles_from_data=True)
    chart2.set_categories(cats_ref2)
    for s in chart2.series:
        if getattr(s.title, "value", None) == "TOTAL":
            s.graphicalProperties.line.solidFill = "000000"
            s.graphicalProperties.line.width = 30000
    chart2.width = 20; chart2.height = 12
    ws.add_chart(chart2, "G27")

    _autofit_columns(ws, first_row=1, last_row=ws.max_row)



print(f"✅ Multi-month report (with 'Pagos a cuenta' counts) saved to: {output_path.resolve()}")

✅ Multi-month report (with 'Pagos a cuenta' counts) saved to: C:\Users\roger\Documents\Codi\Sempreviva\Ingressos\ingresos_report_global.xlsx
